In [6]:
import os
import numpy as np
import json

In [7]:
X = []
Y = []

pre_building_files = sorted(os.listdir('data/train_images/pre_buildings'))
post_building_files = sorted(os.listdir('data/train_images/post_buildings'))

In [17]:
json_source = 'data/train_labels/post_labels/'
post_hurricane_jsons = sorted([f for f in os.listdir(json_source)])

damage_labels = []
for image_json in post_hurricane_jsons:
    with open (json_source + image_json) as f:
        json_data = json.load(f)
        
    for building in json_data['features']['xy']:
        damage_labels.append(building['properties']['subtype'])


In [9]:
from PIL import Image

Images = []
for f in pre_building_files:
    Images.append(Image.open('data/train_images/pre_buildings/' + f).convert('RGB'))

In [10]:
X_pre = []
for I in Images:
    X_pre.append(np.asarray(I))

In [11]:
X_pre = np.array(X_pre)
X_pre.shape

(66092, 50, 50, 3)

In [12]:
Images = []
for f in post_building_files:
    Images.append(Image.open('data/train_images/post_buildings/' + f).convert('RGB'))

In [13]:
X_post = []
for I in Images:
    X_post.append(np.asarray(I))

In [14]:
X_post = np.array(X_post)
X_post.shape

(66092, 50, 50, 3)

In [15]:
X = np.concatenate((X_pre, X_post), axis=3)
X.shape

(66092, 50, 50, 6)

In [18]:
len(damage_labels)

66092

In [19]:
label_map = {'no-damage': 0, 'minor-damage': 1, 'major-damage': 2, 'destroyed': 3, 'un-classified': 4}

for label in damage_labels:
    Y.append(label_map[label])

In [23]:
train_X = X.astype('float32')[:int(0.8*len(X))]
test_X = X.astype('float32')[int(0.8*len(X)):]
train_Y = Y[0:int(0.8*len(Y))]
test_Y = Y[int(0.8*len(Y)):]

52873
13219
52873
13219


In [28]:
print(test_Y.count(0))
print(test_Y.count(1))
print(test_Y.count(2))
print(test_Y.count(3))
print(test_Y.count(4))

9446
2373
983
303
114


In [29]:
# train_X = X.astype('float32')
# train_X /= 255

# train_Y = Y

# train_X.shape

train_X /= 255
test_X /= 255

In [32]:
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.layers.normalization import BatchNormalization
from keras.constraints import max_norm

model = Sequential()

Using TensorFlow backend.


In [33]:
# Define the network
model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='glorot_normal', input_shape=(50, 50, 6)))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='glorot_normal'))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3, 3), padding='same', kernel_initializer='glorot_normal'))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer='glorot_normal'))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# Fully connected layer
model.add(Dense(512, kernel_initializer='glorot_normal'))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, kernel_initializer='glorot_normal'))
model.add(Activation('sigmoid'))


# Defining loss and optimizer
optimizer = RMSprop(lr=1e-4)
loss = 'binary_crossentropy'
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

In [34]:
batch_size = 64
EPOCHS = 5

# test the network:
model.fit(train_X, train_Y, batch_size=batch_size, epochs=EPOCHS, validation_split=0.2, verbose=1, shuffle=True)

Train on 42298 samples, validate on 10575 samples
Epoch 1/5
42298/42298 [==============================] - 230s 5ms/step - loss: -9258.3329 - accuracy: 0.2133 - val_loss: 76643.4283 - val_accuracy: 0.3002
Epoch 2/5
42298/42298 [==============================] - 235s 6ms/step - loss: -537487.1157 - accuracy: 0.2129 - val_loss: 2420065.8081 - val_accuracy: 0.3002
Epoch 3/5
42298/42298 [==============================] - 235s 6ms/step - loss: -6681078.3614 - accuracy: 0.2129 - val_loss: 22243944.1101 - val_accuracy: 0.3002
Epoch 4/5
42298/42298 [==============================] - 262s 6ms/step - loss: -41960990.1185 - accuracy: 0.2129 - val_loss: 112196997.4271 - val_accuracy: 0.3002
Epoch 5/5
42298/42298 [==============================] - 265s 6ms/step - loss: -161281766.6061 - accuracy: 0.2129 - val_loss: 384462397.8818 - val_accuracy: 0.3002


In [38]:
# loss, acc = model.evaluate(train_X, train_Y, batch_size=batch_size, verbose=1)

# print('')
# print('Got %.2f%% accuracy' % (acc * 100.))

test_predict_Y = model.predict(test_X, batch_size=batch_size, verbose=0, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False)
test_predict_Y


array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [45]:
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import accuracy_score

correct_damaged = 0
correct_undamaged = 0
incorrect_damaged = 0
incorrect_undamaged = 0
for index in range(len(test_Y)):
    if int(test_Y[index]) > 0:
        if int(test_predict_Y[index]) > 0:
            correct_damaged += 1
        else:
            incorrect_damaged += 1
    else:
        if int(test_predict_Y[index]) > 0:
            incorrect_undamaged += 1
        else:
            correct_undamaged += 1

print('# of damaged buildings correctly labeled as damaged: ', correct_damaged)
print('# of damaged buildings incorrectly labeled as undamaged: ', incorrect_damaged)
print('# of undamaged buildings correctly labeled as undamaged: ', correct_undamaged)
print('# of undamaged buildings incorrectly labeled as damaged: ', incorrect_undamaged)
    
#multilabel_confusion_matrix(test_Y, test_predict_Y)

# of damaged buildings correctly labeled as damaged:  3773
# of damaged buildings incorrectly labeled as undamaged:  0
# of undamaged buildings correctly labeled as undamaged:  0
# of undamaged buildings incorrectly labeled as damaged:  9446


array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)